In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anxiety_disorder/GSE78104'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"lncRNA and mRNA expression data in peripheral blood sampled from patients with Obsessive-Compulsive Disorder"
!Series_summary	"The aim of the study is to identify the global messenger RNA (mRNA) and long noncoding RNA (lncRNA) expression profiling in peripheral blood from thirty patients with Obsessive Compulsive Disorders (OCD) and thirty paired normal controls."
!Series_overall_design	"We quantified the gene transcripts in peripheral blood from thirty patients with OCD and thirty normal controls by the method of Microarray using Aglilent G3 lncRNA v4.04×180K."
Sample Characteristics Dictionary:
{0: ['tissue: whole blood'], 1: ['disease state: Obsessive-Compulsive Disorder', 'disease state: normal control'], 2: ['gender: male', 'gender: female'], 3: ['age: 25y', 'age: 23y', 'age: 18y', 'age: 26y', 'age: 27y', 'age: 19y', 'age: 22y', 'age: 16y', 'age: 35y', 'age: 32y', 'age: 15y', 'age: 43y', 'age: 36y', 'age: 17y', 'age: 45y', 'age: 40y', 'age: 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True  # Since the dataset involves mRNA and lncRNA expression data

# Step 2: Variable Availability and Data Type Conversion
# Based on the provided Sample Characteristics Dictionary

# Anxiety_disorder (relevant to Obsessive-Compulsive Disorder)
if len(set(['disease state: Obsessive-Compulsive Disorder', 'disease state: normal control'])) > 1:
    trait_row = 1
else:
    trait_row = None

# Age
if len({age.split(': ')[-1] for age in ['age: 25y', 'age: 23y', 'age: 18y', 'age: 26y', 'age: 27y', 'age: 19y', 'age: 22y', 'age: 16y', 'age: 35y', 'age: 32y', 'age: 15y', 'age: 43y', 'age: 36y', 'age: 17y', 'age: 45y', 'age: 40y', 'age: 28y', 'age: 31y', 'age: 60y', 'age: 59y', 'age: 24y', 'age: 20y', 'age: 21y', 'age: 44y', 'age: 37y', 'age: 30y', 'age: 56y']}) > 1:
    age_row = 3
else:
    age_row = None

# Gender
if len(set(['gender: male', 'gender: female'])) > 1:
    gender_row = 2
else:
    gender_row = None

# Data conversion functions
def convert_trait(value):
    actual_value = value.split(': ')[-1]
    if actual_value == "Obsessive-Compulsive Disorder":
        return 1
    elif actual_value == "normal control":
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[-1].rstrip('y'))
    except:
        return None

def convert_gender(value):
    actual_value = value.split(': ')[-1]
    if actual_value == "male":
        return 1
    elif actual_value == "female":
        return 0
    else:
        return None

# Step 3: Save Metadata
save_cohort_info('GSE78104', './preprocessed/Anxiety_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction (since trait_row is not None)
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Anxiety_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Anxiety_disorder/trait_data/GSE78104.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM2067403': [1.0, 25.0, 1.0], 'GSM2067404': [1.0, 23.0, 0.0], 'GSM2067405': [1.0, 18.0, 0.0], 'GSM2067406': [1.0, 26.0, 1.0], 'GSM2067407': [1.0, 27.0, 0.0], 'GSM2067408': [1.0, 19.0, 1.0], 'GSM2067409': [1.0, 22.0, 1.0], 'GSM2067410': [1.0, 27.0, 0.0], 'GSM2067411': [1.0, 18.0, 1.0], 'GSM2067412': [1.0, 25.0, 1.0], 'GSM2067413': [1.0, 16.0, 1.0], 'GSM2067414': [1.0, 35.0, 0.0], 'GSM2067415': [1.0, 16.0, 1.0], 'GSM2067416': [1.0, 16.0, 1.0], 'GSM2067417': [1.0, 32.0, 1.0], 'GSM2067418': [1.0, 18.0, 1.0], 'GSM2067419': [1.0, 15.0, 1.0], 'GSM2067420': [1.0, 43.0, 1.0], 'GSM2067421': [1.0, 36.0, 1.0], 'GSM2067422': [1.0, 17.0, 0.0], 'GSM2067423': [1.0, 45.0, 0.0], 'GSM2067424': [1.0, 40.0, 1.0], 'GSM2067425': [1.0, 35.0, 1.0], 'GSM2067426': [1.0, 28.0, 0.0], 'GSM2067427': [1.0, 27.0, 1.0], 'GSM2067428': [1.0, 31.0, 1.0], 'GSM2067429': [1.0, 23.0, 1.0], 'GSM2067430': [1.0, 35.0, 0.0], 'GSM2067431': [1.0, 60.0, 1.0], 'GSM2067432': [1.0, 59.0, 0.0], 'GSM2067433': [0.0, 24.0, 1.0], 'GSM206

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(-)3xSLv1', 'A_19_P00315459',
       'A_19_P00315492', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315538',
       'A_19_P00315633', 'A_19_P00315668', 'A_19_P00315717', 'A_19_P00315718'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_19_P00315459', 'A_19_P00315492', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315538'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'SEQUENCE': ['AGCCCCCACTGTTCCACTTATTGTGATGGTTTGTATATCTTTATTTCAAAGAAGATCTGT', 'AGGCAGCCTTGCTGTTGGGGGTTATTGGCAGCTGTTGGGGGTTAGAGACAGGACTCTCAT', 'AGCCGGGATCGGGTTGTTGTTAATTTCTTAAGCAATTTCTAAATTCTGTATTGACTCTCT', 'CAATGGATTCCATGTTTCTTTTTCTTGGGGGGAGCAGGGAGGGAGAAAGGTAGAAAAATG', 'CACAATGACCATCATTGAGGGCGATGTTTATGCTTCCATTGTTAGTTTAGATATTTTGTT'], 'TargetID': [nan, 'Q73P46', 'P01115', nan, nan], 'ncRNA_SeqID': [nan, nan, nan, nan, nan], 'Source': ['Agilent_humanG3V2', 'Agilent_humanG3V2', 'Agilent_humanG3V2', nan, nan], 'ncRNA_Accession': [nan, nan, nan, nan, nan], 'Chr': ['chrX', 'chr4', 'chr10', nan, nan], 'Start': [149131107.0, 129376376.0, 6780785.0, nan, nan], 'End': [149131166.0, 129376435.0, 6780844.0, nan, nan], 'strand': ['+', '+', '+', nan, nan], 'Description': [nan, 'Q73P46_TREDE (Q73P46) Branched-chain ami

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys store the identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GeneSymbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Anxiety_disorder/gene_data/GSE78104.csv'
os.makedirs(os.path.dirname(gene_csv_path), exist_ok=True)
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Anxiety_disorder')

# 4. Save the cohort information.
save_cohort_info('GSE78104', './preprocessed/Anxiety_disorder/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Anxiety_disorder/GSE78104.csv'
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Anxiety_disorder', the least common label is '1.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Anxiety_disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 18.75
  50% (Median): 27.0
  75%: 35.0
Min: 15.0
Max: 60.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

